# Pushing and analyzing assembly results

In [11]:
phages=open("./data/K2.151228_PacBio_Nahant_Phages.txt").readlines()
phages=[i.replace("\r\n","") for i in phages]
groups=open("/Users/jmb/Desktop/ViralFate/OrthoMCL/tSNE/tSNE_genome_clusters.txt").readlines()
plist=[i.split("\t")[0] for i in groups[1:]]
grplist=[i.split("\t")[-1].replace("\n","") for i in groups[1:]]
pgd=dict(zip(plist, grplist))
pgd

for p in phages:
    print("%s\t%s" % (p, pgd[p]))

1.080.O.	5
1.043.O.	5
1.095.O.	5
1.107.C.	5
1.020.O.	5
1.008.O.	5
1.249.A.	0
1.190.O.	25
1.259.O.	24
2.159.B.	12
1.028.O.	12
1.257.O.	23
1.082.O.	17
1.176.O.	17
1.228.O.	17
1.148.O.	17
1.233.B.	17
1.189.O.	17
1.182.O.	17
1.074.O.	17
1.165.O.	17
1.154.O.	15
1.211.B.	27
1.110.O.	17
1.275.O.	28
1.131.O.	17
1.239.O.	17
1.111.B.	21
1.216.O.	22
12G01.O.	1
1.054.O.	9
1.090.B.	1
1.135.O.	22
1.236.O.	22
1.052.A.	15
1.209.O.	8
1.185.O.	10
1.291.O.	14
1.077.O.	18
1.204.O.	16
1.151.O.	22
1.276.O.	22
1.144.O.	22
1.009.O.	3
1.122.B.	7
1.231.O.	22
2.095.B.	22
2.096.O.	15
2.044.O.	2
1.177.O.	22
1.087.A.	3
1.030.O.	3
1.123.O.	4
1.029.O.	9
1.253.O.	32
1.262.O.	30
1.210.O.	19
1.293.O.	30
1.191.O.	3
1.007.O.	4
1.042.O.	9
1.181.O.	3
1.086.O.	3
2.117.O.	29
1.232.O.	22
1.205.O.	28
1.285.O.	6
1.283.C.	6
1.261.O.	26
2.130.O.	11
1.215.A.	31
3.058.O.	34
1.063.O.	16
1.193.O.	16
1.187.O.	16
1.170.O.	16
1.161.O.	16
1.084.O.	20
1.121.O.	16
1.031.O.	13
1.255.O.	13
2.275.O.	13


In [14]:
from collections import defaultdict
pbonly=open("./data/asm1_blast_comparison_summed_matches.txt").readlines()

pbodict=defaultdict(lambda:set())

for l in pbonly[1:]:
    vec=l.split("\t")
    ctg=vec[0]
    phage=vec[1]
    pbodict[phage].add(ctg)

defaultdict(<function __main__.<lambda>>,
            {'1.007.O.': {'ctg7180000000098'},
             '1.028.O.': {'ctg7180000000156'},
             '1.029.O.': {'ctg7180000000158'},
             '1.030.O.': {'ctg7180000000154'},
             '1.042.O.': {'ctg7180000000109'},
             '1.052.A.': {'ctg7180000000159'},
             '1.054.O.': {'ctg7180000000121'},
             '1.063.O.': {'ctg7180000000088'},
             '1.074.O.': {'ctg7180000000127', 'ctg7180000000142'},
             '1.084.O.': {'ctg7180000000091'},
             '1.086.O.': {'ctg7180000000141'},
             '1.087.A.': {'ctg7180000000150'},
             '1.090.B.': {'ctg7180000000126'},
             '1.110.O.': {'ctg7180000000118'},
             '1.121.O.': {'ctg7180000000086'},
             '1.122.B.': {'ctg7180000000117'},
             '1.123.O.': {'ctg7180000000107'},
             '1.131.O.': {'ctg7180000000120'},
             '1.135.O.': {'ctg7180000000108'},
             '1.144.O.': {'ctg7180000000111'}

In [ ]:
from collections import defaultdict
import os
from pyfaidx import Fasta

if os.path.exists("./contigs/")==False:
    os.mkdir("./contigs/")

phages=open("K2.151228_PacBio_Nahant_Phages.txt").readlines()
phages=[i.replace("\r\n","") for i in phages]

pbonly=open("asm1_blast_comparison_summed_matches.txt").readlines()

pbodict=defaultdict(lambda:set())

for l in pbonly[1:]:
    vec=l.split("\t")
    ctg=vec[0]
    phage=vec[1]
    pbodict[phage].add(ctg)
    
kkhgap=open("kkhgap_blast_comparison_summed_matches.txt").readlines()
kkdict=defaultdict(lambda:set())

for k in kkhgap[1:]:
    vec=k.split("\t")
    ctg=vec[0]
    phage=vec[1]
    ct=float(vec[-1].replace("\n",""))
    if ct<0.15:
        kkdict[phage].add(ctg)

for p in phages:
    try1="./try1/nvp_%s_binned_asm_try1/" % p
    try2="./nvp_%s_binned_asm_try2/" % p
    if os.path.exists(try1) or os.path.exists(try2) or p in pbodict.keys():
        print "some %s assemblies exist" % p
        
        hybrids=["%s9-terminator/asm.ctg.fasta" % try1, "%s9-terminator/asm.ctg.fasta" % try2]
        if os.path.exists("./contigs/%s" % p)==False:
            os.mkdir("./contigs/%s" % p)
        outfasta=open("./contigs/%s/%sasm.tries.fasta" % (p,p), "w")
        for h in hybrids:
            if os.path.exists(h):
                print "%s worked" % h
                f1=Fasta(h)
                for f in f1.keys():
                    single=open("./contigs/%s/%s%s.hybrid.asm.fasta" % (p, p, h.split("/")[1]),"w")
                    single.write(">%s%s\n%s\n" % (p, h.split("/")[1], str(f1[f])))
                    outfasta.write(">%s%s\n%s\n" % (p, h.split("/")[1], str(f1[f])))
                    single.close() 
        pbonlys=[("./nvp_pba_1/9-terminator/asm.ctg.fasta","pbcr1", pbodict), ("/nobackup1/jbrown/nahant_phage_pacbio/SMRTportal_HGAPv2_defaults/kkHGAP.fasta","kkhgap1",kkdict)]
        for assembly, name, cdict in pbonlys:
            f2=Fasta(assembly)
            ctgs=cdict[p]
            if len(ctgs)>0:
                print "%s has a match in the original %s assembly" % (p, name)
                for c in ctgs:
                    single=open("./contigs/%s/%s_%s.fasta" % (p, p, name),"w")
                    single.write(">%s_%s\n%s\n" % (c, name,str(f2[c])))
                    outfasta.write(">%s_%s\n%s\n" % (c, name,str(f2[c])))
                    single.close()
        outfasta.close()

>### Ran above on server using ipython

# Next

## draw gepard dot plots of illumina genomes versus kkhgap assemblies

## Then draw gepard dot plots of the hybrid assemblies

>### on engaging run in ipython:


In [ ]:
from collections import defaultdict
import os
import glob
import subprocess


if os.path.exists("./comparisons/")==False:
    os.mkdir("./comparisons/")
    os.mkdir("./comparisons/gepard/")

phages=[i.replace("\r\n","") for i in open("K2.151228_PacBio_Nahant_Phages.txt").readlines()]

for phage in phages:
    illumina_asm="/nobackup1/jbrown/annotation/genomes/%sfinal.fasta" % phage
    if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%s_kkhgap1.fasta" % (phage, phage)):
        kkhgap=glob.glob("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/*kkhgap1.fasta" % phage)[0]
    
        output="/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/%s_illumina_vs_kkhgap.png" % phage
        gepard_cmd="bash /nobackup1/jbrown/programs/gepard-1.30/gepardcmd.sh -seq1 %s -seq2 %s -matrix /nobackup1/jbrown/programs/gepard-1.30/matrices/edna.mat -outfile %s" % (illumina_asm, kkhgap, output)
        subprocess.call(gepard_cmd.split(" "))
        print "%s comparison done" % phage
    

In [ ]:
for phage in phages:
    illumina_asm="/nobackup1/jbrown/annotation/genomes/%sfinal.fasta" % phage
    if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%stry1.hybrid.asm.fasta" % (phage, phage)):
        kkhgap=glob.glob("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%stry1.hybrid.asm.fasta" % (phage,phage))[0]
    
        output="/nobackup1/jbrown/nahant_phage_pacbio/comparisons/gepard/%s_illumina_vs_try1hybrid.png" % phage
        gepard_cmd="bash /nobackup1/jbrown/programs/gepard-1.30/gepardcmd.sh -seq1 %s -seq2 %s -matrix /nobackup1/jbrown/programs/gepard-1.30/matrices/edna.mat -outfile %s" % (illumina_asm, kkhgap, output)
        subprocess.call(gepard_cmd.split(" "))
        print "%s comparison done" % phage

## Identification of cut sites for pacbio/hgap contigs that were larger than their illumina counterparts

> I want to cut the pacbio/hgap contigs at the start location of the illumina contigs

In [8]:
blast=open("./data/kkhgap_vs_igenomes.txt").readlines()
print len(blast)

goodblast=[]

for l in blast:
    vec=l.split("\t")
    if float(vec[2])>99 and int(vec[3])>1000:
        goodblast.append(l)
print len(goodblast)

7076
144


In [15]:
from pyfaidx import Fasta
#create text object containing information about pb assembly contig lengths:

kkasm=Fasta("./data/kkHGAP.fasta") 

kk_lens={}

for s in kkasm.keys():
    length=len(str(kkasm[s]))
    kk_lens[s]=length
    
ilens=open("./data/igenome_lengths.txt").readlines()
phages=[i.split("\t")[0] for i in ilens[1:]]
lens=[i.split("\t")[1].replace("\n","") for i in ilens[1:]]
i_lens=dict(zip(phages, lens))

In [48]:
from collections import defaultdict
starts=""
starts=defaultdict(lambda: (200000,200000))

#identifying earliest point on target contig where query hits, the corresponding match on the query contig is where we will cut it for circularization
for l in goodblast:
    vec=l.split("\t")
    q=vec[0]
    t=vec[1].split("_")[1]
    if int(kk_lens[q])>int(i_lens[t]):
        qstart=vec[6]
        tstart=vec[8]
        if int(tstart)<starts[t][1]:
            #print "changing %s to start %s instead of %s" % (t, tstart, starts[q][1])
            starts[t]=(int(qstart), int(tstart))

for l in goodblast:
    vec=l.split("\t")
    q=vec[0]
    t=vec[1].split("_")[1]
    qstart=vec[6]
    qstop=vec[7]
    tstart=vec[8]
    if int(kk_lens[q])>int(i_lens[t]) and abs(int(kk_lens[q])-int(i_lens[t]))>200:
        if starts[t][1]==1:
            continue
        elif starts[t][0]==1:
            starts[t]=(int(qstop),int(tstart))

starts

out=open("./data/kkhgap_cut_spots.txt","w")
for s in starts.keys():
    out.write("%s\t%s\n" % (s, starts[s][0]))
out.close()

In [49]:
!head ./data/kkhgap_cut_spots.txt

2.096.O.	40035
1.211.B.	31216
1.007.O.	48843
2.117.O.	35838
1.161.O.	874
1.144.O.	26852
1.054.O.	34110
1.190.O.	19002
2.130.O.	83927
1.193.O.	82697


### Create split contig files for recircularization
> ### run in ipython on engaging

In [ ]:
from pyfaidx import Fasta
import os

if os.path.exists("./contigs/hgap1_to_circularize/")==False:
    os.mkdir("./contigs/hgap1_to_circularize/")

spots=open("kkhgap_cut_spots.txt").readlines()
starts=dict(zip([i.split("\t")[0] for i in spots], [int(i.split("\t")[1].replace("\n","")) for i in spots]))

for phage in starts.keys():
    if os.path.exists("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%s_kkhgap1.fasta" % (phage, phage)):
        contig=Fasta("/nobackup1/jbrown/nahant_phage_pacbio/contigs/%s/%s_kkhgap1.fasta" % (phage, phage))
        split1=str(contig[0][0:starts[phage]])
        split2=str(contig[0][starts[phage]:])
        out=open("./contigs/hgap1_to_circularize/%shgap.to_circularize.fasta" % phage, "w")
        out.write(">%s\n%s\n>%s_1\n%s" % (contig[0].name,split1,contig[0],split2))
        out.close()
